# VacationPy
----




------------------------------------
- This code took about 18 seconds to run
- i've ran it a couple times and there seems to be a bias around africa, the middle east, and india

In [105]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# from full_country import full_name

### this allows me to keep my keys in a folder outside my repo
import sys
try:
    sys.path.insert(1,'/Users/Trav/Desktop/Coding/GitHub_Repositories/API_PACKAGE')
    from api_keys import g_key
except:
    from api_keys import g_key





{'AD': 'Andorra',
 'AE': 'United Arab Emirates',
 'AF': 'Afghanistan',
 'AG': 'Antigua & Barbuda',
 'AI': 'Anguilla',
 'AL': 'Albania',
 'AM': 'Armenia',
 'AN': 'Netherlands Antilles',
 'AO': 'Angola',
 'AQ': 'Antarctica',
 'AR': 'Argentina',
 'AS': 'American Samoa',
 'AT': 'Austria',
 'AU': 'Australia',
 'AW': 'Aruba',
 'AZ': 'Azerbaijan',
 'BA': 'Bosnia and Herzegovina',
 'BB': 'Barbados',
 'BD': 'Bangladesh',
 'BE': 'Belgium',
 'BF': 'Burkina Faso',
 'BG': 'Bulgaria',
 'BH': 'Bahrain',
 'BI': 'Burundi',
 'BJ': 'Benin',
 'BM': 'Bermuda',
 'BN': 'Brunei Darussalam',
 'BO': 'Bolivia',
 'BR': 'Brazil',
 'BS': 'Bahama',
 'BT': 'Bhutan',
 'BU': 'Burma (no longer exists)',
 'BV': 'Bouvet Island',
 'BW': 'Botswana',
 'BY': 'Belarus',
 'BZ': 'Belize',
 'CA': 'Canada',
 'CC': 'Cocos (Keeling) Islands',
 'CF': 'Central African Republic',
 'CG': 'Congo',
 'CH': 'Switzerland',
 'CI': "Côte D'ivoire (Ivory Coast)",
 'CK': 'Cook Iislands',
 'CL': 'Chile',
 'CM': 'Cameroon',
 'CN': 'China',
 'CO': 

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [95]:
## import csv to dataframe, add try block to deal with directory issues
csv_import_path = os.path.join('..','Weather','Export Folder','city_data.csv')
city_weather_df=pd.read_csv(csv_import_path)

city_weather_df.head()

# # ### using this to limit the request, remove once completed
# city_weather_df=city_weather_df.head()
# city_weather_df.head(20)

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
0,-60.96,-19.60,cidreira,br,"(-30.183333, -50.2)",-30.183333,-50.200000,63.39,68,81,8.88,63.39
1,14.92,123.07,luklukan,ph,"(14.3254, 122.7124)",14.325400,122.712400,83.89,71,100,18.81,83.89
2,-29.07,-102.83,lebu,cl,"(-37.616667, -73.65)",-37.616667,-73.650000,51.26,90,3,12.26,51.26
3,32.52,-79.77,mount pleasant,us,"(32.7938889, -79.8627778)",32.793889,-79.862778,60.48,72,1,14.32,62.60
4,0.83,-121.17,atuona,pf,"(-9.8, -139.0333333)",-9.800000,-139.033333,77.72,76,64,11.43,77.72


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [96]:
#configure gmaps with key
gmaps.configure(api_key=g_key)

In [97]:

#defines coordinates to plot and weight is humididty
### shoul these be .astype(float)
locations=city_weather_df[['Latitude','Longitude']]
weighty=city_weather_df['Humidity']

fig=gmaps.figure(center=(0,0),zoom_level=2)
heat=gmaps.heatmap_layer(locations,weights=weighty,dissipating=True,max_intensity=200,point_radius=20)
fig.add_layer(heat)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [98]:
#narrows down the dataframe to a dataframe with ideal weather
ideal_weather = city_weather_df.loc[(city_weather_df['Max Temp']>70)&(city_weather_df['Max Temp']<80)&(city_weather_df['Wind Speed']<10)&(city_weather_df['Cloudiness']==0),:]

#drops bad data
ideal_weather.dropna()


ideal_weather.head()

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
7,-21.52,-128.23,rikitea,pf,"(-23.1202778, -134.9691667)",-23.120278,-134.969167,76.10,64,0,2.66,76.1
135,13.13,61.75,salalah,om,"(17.0175, 54.0827778)",17.017500,54.082778,73.40,38,0,3.36,73.4
140,-0.77,-55.07,alenquer,br,"(-1.933333, -54.766667)",-1.933333,-54.766667,77.00,94,0,3.36,77.0
184,20.43,68.30,porbandar,in,"(21.633333, 69.6)",21.633333,69.600000,77.00,53,0,8.52,77.0
188,26.05,-23.21,los llanos de aridane,es,"(28.658507, -17.918211)",28.658507,-17.918211,71.24,43,0,9.17,71.6


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [99]:
#simplifies and defines the hotel_df
hotel_df=ideal_weather.drop(['Approximate Lat','Approximate Lon','Exact Coordinates'],axis=1).reset_index(drop=True)

#creates blank series that will be filled in
hotel_df['Hotel Name']=""
hotel_df['Hotel Lat']=""
hotel_df['Hotel Lon']=""
hotel_df=hotel_df.rename(columns={'City Name':'City','Country Code':'Country'})
hotel_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,rikitea,pf,-23.120278,-134.969167,76.10,64,0,2.66,76.1,,,
1,salalah,om,17.017500,54.082778,73.40,38,0,3.36,73.4,,,
2,alenquer,br,-1.933333,-54.766667,77.00,94,0,3.36,77.0,,,
3,porbandar,in,21.633333,69.600000,77.00,53,0,8.52,77.0,,,
4,los llanos de aridane,es,28.658507,-17.918211,71.24,43,0,9.17,71.6,,,


In [100]:

### finds the closest hotel and fills in the hotel name and coordinates to the dataframe
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_radius = 5000
target_type = "Hotel"

for i,r in hotel_df.iterrows():
    c_lat=hotel_df['Latitude'][i]
    c_lon = hotel_df['Longitude'][i]
    target_coordinates =f'{c_lat}, {c_lon}' 
    
    param = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    places_json=requests.get(base_url,params=param).json()
    ## the city tessalit is literally in the middle of the sahara desert and returns no results, so this is a work around.

    if places_json['status']=='ZERO_RESULTS':
        hotel_df['Hotel Name'][i]=np.nan
    else:
        hotel_df['Hotel Name'][i]=places_json["results"][0]['name']
        hotel_df['Hotel Lat'][i]=places_json["results"][0]['geometry']['location']['lat']
        hotel_df['Hotel Lon'][i]=places_json["results"][0]['geometry']['location']['lng']


## Dropping the cities that returned no results 
hotel_df=hotel_df.dropna()

hotel_df.head(30)


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,rikitea,pf,-23.120278,-134.969167,76.10,64,0,2.66,76.10,Pension Maro'i,-23.1206,-134.969
1,salalah,om,17.017500,54.082778,73.40,38,0,3.36,73.40,HYATT SALALAH HOTEL,17.0194,54.1108
2,alenquer,br,-1.933333,-54.766667,77.00,94,0,3.36,77.00,Hotel Pepita,-1.95706,-54.738
3,porbandar,in,21.633333,69.600000,77.00,53,0,8.52,77.00,Lords Inn Porbandar,21.6314,69.609
4,los llanos de aridane,es,28.658507,-17.918211,71.24,43,0,9.17,71.60,Hotel Hacienda de Abajo,28.6433,-17.9349
5,barao de melgaco,br,-16.216667,-55.966667,78.80,69,0,3.36,78.80,Tuiuiú Pantanal Hotel,-16.2009,-55.9745
6,rajgurunagar,in,18.866667,73.900000,77.04,48,0,3.60,77.04,SPOT ON 64658 Hotel Yuvraj Executive,18.9084,73.9156
7,malumfashi,ng,11.785288,7.621753,71.35,31,0,7.36,71.35,Malumfashi Motel,11.7768,7.61042
8,puerto del rosario,es,28.500380,-13.862719,71.60,68,0,8.05,71.60,Hotel El Mirador,28.4738,-13.8657
9,umm lajj,sa,25.021264,37.268502,77.29,34,0,4.45,77.29,Shada Collection Mina Hotel,25.042,37.2526


In [101]:
# # #create a new folder and exports the dataframe

def dataframe_to_csv(df):
    current_directory=os.getcwd()
    new_folder_path =os.path.join(current_directory,r'Export Folder')
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    csv_export_path = os.path.join(new_folder_path,'hotel_data.csv')
    df.to_csv(csv_export_path,index=False)
dataframe_to_csv(hotel_df)

In [102]:
## import the csv file you just saved
def import_path():
    current_directory=os.getcwd()
    import_path.new_folder_path =os.path.join(current_directory,r'Export Folder')
    csv_export_path = os.path.join(import_path.new_folder_path,'hotel_data.csv')
    return csv_export_path

im_path=import_path()
csv_imp_df=pd.read_csv(im_path,delimiter=',')
csv_imp_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,rikitea,pf,-23.120278,-134.969167,76.10,64,0,2.66,76.1,Pension Maro'i,-23.120620,-134.968753
1,salalah,om,17.017500,54.082778,73.40,38,0,3.36,73.4,HYATT SALALAH HOTEL,17.019380,54.110750
2,alenquer,br,-1.933333,-54.766667,77.00,94,0,3.36,77.0,Hotel Pepita,-1.957062,-54.738019
3,porbandar,in,21.633333,69.600000,77.00,53,0,8.52,77.0,Lords Inn Porbandar,21.631415,69.609035
4,los llanos de aridane,es,28.658507,-17.918211,71.24,43,0,9.17,71.6,Hotel Hacienda de Abajo,28.643326,-17.934936


In [103]:


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in csv_imp_df.iterrows()]

# hotel_info


In [104]:
### plotting heat map and hotel locations
y_locations=city_weather_df[['Latitude','Longitude']]
x_locations=csv_imp_df[['Hotel Lat','Hotel Lon']]
x_weighty=city_weather_df['Humidity']



x_fig=gmaps.figure(center=(0,0),zoom_level=2)
x_heat=gmaps.heatmap_layer(y_locations,weights=x_weighty,dissipating=True,max_intensity=200,point_radius=20)
x_marks=gmaps.marker_layer(x_locations,info_box_content=hotel_info)
x_fig.add_layer(x_marks)
x_fig.add_layer(x_heat)

x_fig

Figure(layout=FigureLayout(height='420px'))